# spam
https://www.kaggle.com/uciml/sms-spam-collection-dataset

拿到数据首先读入拿到数据

In [1]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

data_dir = "../input/"
df = pd.read_csv(data_dir + '/spam.csv', encoding='latin-1')
X_train, X_test, y_train, y_test = train_test_split(
    df.v2,
    df.v1, 
    test_size=0.2, 
    random_state=0)  



df.shape





(5572, 5)

In [2]:
def GetVocabulary(data):
    vocab_dict={}
    word_id=0
    for document in data:
        words=document.split()
        for word in words:
            word=word.lower()
            if word not in vocab_dict:
                vocab_dict[word]=word_id
                word_id+=1
    return vocab_dict

vocab=GetVocabulary(X_train)
vocab.get('ok')
len(vocab)

11706

In [3]:
# convert document to vector
def document2vector(vocab,document):
    word_vector=np.zeros(len(vocab))
    words=document.split()
    not_in_vocab=0
    for word in words:
        word=word.lower()
        if word in vocab:
            word_vector[vocab[word]]+=1
        else:
            not_in_vocab+=1
    return word_vector, not_in_vocab
            
word_vector2,not_in_cab=document2vector(vocab,'here it is')
print(word_vector2)
vocab.get('is')


[0. 0. 1. ... 0. 0. 0.]


2

In [4]:
# change to vector
Xtrain_vector=[]
for document in X_train.values:
    word_vector,not_in_vocab=document2vector(vocab, document)
    Xtrain_vector.append(word_vector)
    
    
print(len(Xtrain_vector))
print(len(Xtrain_vector[7])) 
print(len(y_train))
print(y_train[3])
print(Xtrain_vector[6])
print(y_train[8])

print(Xtrain_vector[8].sum())

4457
11706
4457
ham
[0. 0. 0. ... 0. 0. 0.]
spam
22.0


In [5]:
y_train2=pd.DataFrame(y_train)
y_train2.head()
ham_number,spam_number= y_train2.groupby('v1').size()
p_ham=np.log(ham_number/(ham_number+spam_number))
p_spam=np.log(spam_number/(ham_number+spam_number))
print(p_ham,p_spam,y_train.size)


-0.13967219993589897 -2.0374804160946445 4457


In [6]:

def NaiveBayes(Xtrain_vector,y_train):
    num_docs=len(Xtrain_vector)
    print(num_docs)
    num_words=len(Xtrain_vector[0])
    print(num_words)
    
    ham_word_counter=np.ones(num_words)
    spam_word_counter=np.ones(num_words)
    
    total_ham_word_counter=0
    total_spam_word_counter=0
    print(Xtrain_vector[0])
    
    for i in range(num_docs):
        
        if y_train[i]=='ham':
            total_ham_word_counter+=sum(Xtrain_vector[i])
            ham_word_counter+=Xtrain_vector[i]
        
        else:
            total_spam_word_counter+=sum(Xtrain_vector[i])
            spam_word_counter+=Xtrain_vector[i]
            
            
    p_spam_vector=np.log(spam_word_counter/(total_spam_word_counter+num_words))
    p_ham_vector=np.log(ham_word_counter/(total_ham_word_counter+num_words))

    return p_spam_vector, p_ham_vector,total_spam_word_counter,total_ham_word_counter,num_words


p_spam_vector, p_ham_vector,total_spam_word_counter,total_ham_word_counter,num_words=NaiveBayes(Xtrain_vector,y_train.values)

    
    

4457
11706
[1. 1. 2. ... 0. 0. 0.]


navie bayes training

In [7]:
def prediction_bayes(X_test_vector, p_spam_vector, p_ham_vector,p_spam,p_ham,spam_smoothing, ham_smoothing):
    spam =sum(X_test_vector* p_spam_vector) + p_spam + spam_smoothing
    ham = sum(X_test_vector * p_ham_vector) + p_ham + ham_smoothing
    if spam > ham:
        print(spam)
        return 'spam'
    else:
        return 'ham'
        

predictions=[]

for document in X_test.values:
    X_test_vector,not_in_vocb=document2vector(vocab,document)
    # Add smoothing for out_of_vocbulary words
    if not_in_vocb != 0:
        spam_smoothing = np.log(not_in_vocb/(total_spam_word_counter + num_words))
        ham_smoothing = np.log(not_in_vocb/(total_ham_word_counter + num_words))
    else:
        spam_smoothing = 0
        ham_smoothing = 0
    ans = prediction_bayes(X_test_vector, p_spam_vector, p_ham_vector,p_spam,p_ham,spam_smoothing, ham_smoothing)
    predictions.append(ans)


-173.78319857384545
-199.1828324881395
-104.98997617691708
-121.58339118834192
-195.56979161410146
-66.06061843164161
-179.587450197175
-112.593594975299
-199.04761861500313
-112.84099989124243
-112.8818322286321
-199.5665994778352
-176.78744440358895
-167.62003516475085
-137.01464217769086
-167.9195967745867
-188.51849059253766
-129.83162618663948
-154.754203912541
-135.6534670051044
-202.4195662096861
-180.73938688522387
-197.44137734424984
-187.90736709975795
-177.42676975887198
-201.9714814417114
-178.64549764346393
-172.41584063127036
-49.453203592735335
-179.38634351753063
-139.93958206944384
-161.43248968679876
-178.33632745830823
-138.48030537371426
-215.0603664428527
-91.52123837265654
-148.8247238068553
-154.3931499360932
-190.96190806059673
-56.081439472251205
-123.66272928909189
-61.36599774049385
-211.51254548399373
-161.1586394476845
-56.081439472251205
-168.13810743785461
-114.24122181387636
-171.4135901784928
-198.76485554524615
-155.42669354624687
-175.0609959499002
-1

In [8]:

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score


print (accuracy_score(predictions, y_test))
print (classification_report(y_test, predictions))
print (confusion_matrix(y_test, predictions))

0.97847533632287
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       949
        spam       0.98      0.87      0.92       166

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

[[946   3]
 [ 21 145]]
